## 5. Planning & Execution
 - simple test scenario: real world single object P&P
* **5.1 Indy**
* **5.2 Panda**
* **5.3 Indy&Panda**

### Prepare the task scene
* Prepare Indy and Panda. Install gripper to indy.
    - If gripper is not prepared, just remove endtool from Indy and assume gripper is there
* Prepare "floor", "track", "goal", "box1" objects as defined in *pkg.detector.aruco.marker_config.py*
* The robots and "track" are installed on "floor".
* "goal" and "box1" are on "track"
* In this scenario, "box1" will be moved onto "goal" plate
<img src="./Figs/5.PlanningAndExecution.jpg" width="80%">

## 5.1 Indy

##### initialize CombinedRobot and GeometryScene
* Prepare the robots as instructed in release/1.Robots
* Connect gripper to indy - match the direction of grippper with the model added below
* Prepare Kinect and Realsense

In [1]:
import os
import sys
import numpy as np
import time
sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

from pkg.controller.combined_robot import CombinedRobot
from pkg.controller.robot_config import RobotConfig, RobotType
INDY_IP = "192.168.21.6"
PANDA_HOST_IP = "192.168.21.14" ## The host computer is the computer that runs control program
PANDA_ROBOT_IP = "192.168.21.13" ## The robot has it's own IP

crob = CombinedRobot(
    robots_on_scene=[
        RobotConfig(0, RobotType.indy7gripper, ((0,-0.3,0), (0,0,0)),
                    INDY_IP),
#         RobotConfig(1, RobotType.panda, ((0,0.3,0), (0,0,0)),
#                     "{}/{}".format(PANDA_HOST_IP, PANDA_ROBOT_IP))
    ], 
    connection_list=[False, False])

from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.aruco.marker_config import get_aruco_map
from pkg.detector.camera.kinect import Kinect
from pkg.detector.camera.realsense import RealSense
from pkg.detector.detector_interface import DetectionLevel

stereo = ArucoStereo(aruco_map=get_aruco_map(), 
                     camera_list=[Kinect(), RealSense()])

stereo.initialize()

time.sleep(1) # Let the camera have some time to get stable
stereo.calibrate()
print("Detector initialized")

from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder.instance(detector=stereo)
# deprecated: s_builder.reset_reference_coord(ref_name="floor")

xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
ptems = s_builder.add_poles(
    {"cam0": stereo.ref_coord_inv[:3,3], 
     "cam1":np.matmul(stereo.ref_coord_inv, stereo.T_c12)[:3,3]},
    color=(0.6,0.6,0.6,0.2))

gscene.show_pose(crob.home_pose)

connection command:
indy0: False
Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming
Detector initialized
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscribe

##### add geometries

In [2]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])
track = gtem_dict["track"]

gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])
s_builder.give_clearance(track, gtem_dict.values())

##### create PlanningScene

In [3]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

##### create binders

In [4]:
from pkg.geometry.geometry import *
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp",
                   dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0),
                   color=(1,0,0,1), display=True, collision=False, fixed=True)

pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="track", gname="track", _type=PlacePlane, point=None)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=(0,0,0.005))

##### create_subject

In [5]:
from pkg.planning.constraint.constraint_subject import Grasp2Point, PlacePoint
from pkg.planning.constraint.constraint_subject import CustomObject, BoxObject

In [6]:
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=BoxObject)

##### create PlanningPipeline

In [7]:
from pkg.planning.pipeline import PlanningPipeline
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.task.rrt import TaskRRT

ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(MoveitPlanner(pscene))
ppline.set_task_planner(TaskRRT(pscene))

##### update state

In [9]:
initial_state = pscene.initialize_state(crob.home_pose)
print(initial_state.node)

('track',)


##### search for a plan

In [10]:
ppline.search(initial_state, goal_nodes=[("goal",)], verbose=True, 
              display=False, dt_vis=0.01, timeout_loop=100, multiprocess=True, timeout=1, timeout_constrained=10)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
ppline.play_schedule(schedule, period=0.05)

try: 0 - ('track',)->('grip0',)
error: 2.5
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('track',)->('grip0',) = fail
try: 0 - ('track',)->('grip0',)
error: 2.7
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('track',)->('grip0',) = fail
try: 0 - ('track',)->('grip0',)
error: 2.9
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('track',)->('grip0',) = fail
try: 0 - ('track',)->('grip0',)
error: 2.7
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('track',)->('grip0',) = fail
try: 0 - ('track',)->('grip0',)
error: 0.77
try transition motion
transition motion tried: True
result: 0 - ('track',)->('grip0',) = success
branching: 0->1 (4.24/100.0 s, steps/err: 33(78.7189006805 ms)/0.00216334934249)
try: 1 - ('grip0',)->('goal',)
error: 1.5
try transition motion
transition motion tried: True
Goal reached
result: 1 - ('grip0',)->('goal',) = suc

result: 14 - ('grip0',)->('goal',) = fail
try: 6 - ('track',)->('grip0',)
error: 0.77
try transition motion
transition motion tried: True
result: 6 - ('track',)->('grip0',) = success
branching: 6->15 (27.66/100.0 s, steps/err: 13(49.3750572205 ms)/0.00150748763961)
try: 15 - ('grip0',)->('goal',)
error: 2.0
try transition motion
transition motion tried: False
Motion Plan Failure
result: 15 - ('grip0',)->('goal',) = fail
try: 15 - ('grip0',)->('goal',)
error: 2.8
try transition motion
transition motion tried: False
Motion Plan Failure
result: 15 - ('grip0',)->('goal',) = fail
try: 1 - ('grip0',)->('track',)
error: 2.3
try transition motion
transition motion tried: False
Motion Plan Failure
result: 1 - ('grip0',)->('track',) = fail
try: 1 - ('grip0',)->('goal',)
error: 2.7
try transition motion
transition motion tried: False
Motion Plan Failure
result: 1 - ('grip0',)->('goal',) = fail
try: 10 - ('track',)->('grip0',)
error: 1.9
try transition motion
transition motion tried: True
result: 

##### execute plan

In [11]:
crob.reset_connection(True)

connection command:
indy0: True


* convert waypoint schedule to a safe interpolated schedule

In [12]:
safe_schedule = calculate_safe_schedule(pscene, schedule, vel_lims=0.2, acc_lims=0.5)

* execute safe schedule

In [13]:
ppline.execute_schedule(safe_schedule)

Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)


True

## 5.2 Panda

##### initialize CombinedRobot and GeometryScene
* Prepare the robots as instructed in release/1.Robots
* Prepare Kinect and Realsense

In [8]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

from pkg.controller.combined_robot import CombinedRobot
from pkg.controller.robot_config import RobotConfig, RobotType
INDY_IP = "192.168.21.6"
PANDA_HOST_IP = "192.168.21.14" ## The host computer is the computer that runs control program
PANDA_ROBOT_IP = "192.168.21.13" ## The robot has it's own IP

crob = CombinedRobot(
    robots_on_scene=[
#         RobotConfig(0, RobotType.indy7, ((0,-0.3,0), (0,0,0)),
#                     INDY_IP),
        RobotConfig(1, RobotType.panda, ((0,0.3,0), (0,0,0)),
                    "{}/{}".format(PANDA_HOST_IP, PANDA_ROBOT_IP))
    ], 
    connection_list=[False, False])

xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
ptems = s_builder.add_poles(
    {"cam0": stereo.ref_coord_inv[:3,3], 
     "cam1":np.matmul(stereo.ref_coord_inv, stereo.T_c12)[:3,3]},
    color=(0.6,0.6,0.6,0.2))

gscene.show_pose(crob.home_pose)

connection command:
panda1: False
rospy.init_node() has already been called with different arguments: ('task_planner', ['/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py', '-f', '/home/rnb/.local/share/jupyter/runtime/kernel-2e9edae0-1100-47f3-83a1-c67b02756b91.json'], True, None, False, True)
publication OK
published: [0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


##### add geometries

In [9]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])
track = gtem_dict["track"]

gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])
s_builder.give_clearance(track, gtem_dict.values())

##### create PlanningScene

In [10]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

##### create binders

In [11]:
from pkg.geometry.geometry import *
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

pscene.create_binder(bname="grip1", gname="grip1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="track", gname="track", _type=PlacePlane, point=None)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=(0,0,0.005))

##### create_subject

In [12]:
from pkg.planning.constraint.constraint_subject import Grasp2Point, PlacePoint
from pkg.planning.constraint.constraint_subject import CustomObject, BoxObject

In [13]:
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=BoxObject)

##### create PlanningPipeline

In [14]:
from pkg.planning.pipeline import PlanningPipeline
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.task.rrt import TaskRRT

ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(MoveitPlanner(pscene))
ppline.set_task_planner(TaskRRT(pscene))

##### update state

In [15]:
initial_state = pscene.initialize_state(crob.home_pose)
print(initial_state.node)

('track',)


##### search for a plan

In [16]:
ppline.search(initial_state, goal_nodes=[("goal",)], verbose=True, 
              display=False, dt_vis=0.01, timeout_loop=100, multiprocess=True, timeout=1, timeout_constrained=10)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
ppline.play_schedule(schedule, period=0.05)

Use 18/36 agents
try: 0 - ('track',)->('grip1',)
error: 2.8
try: 0 - ('track',)->('grip1',)
try: 0 - ('track',)->('grip1',)
try transition motion
error: 2.5
error: 2.7
try transition motion
try transition motion
try: 0 - ('track',)->('grip1',)
error: 2.9
try: 0 - ('track',)->('grip1',)
try transition motion
try: 0 - ('track',)->('grip1',)
error: 2.4
error: 2.4
try transition motion
try transition motion
try: 0 - ('track',)->('grip1',)
error: 2.7
try: 0 - ('track',)->('grip1',)
try transition motion
error: 1.3
try transition motion
try: 0 - ('track',)->('grip1',)
transition motion tried: True
try: 2 - ('grip1',)->('goal',)
error: 2.2
branching: 0->2 (0.74/100.0 s, steps/err: 33(167.951107025 ms)/0.00105210148171)
result: 0 - ('track',)->('grip1',) = success
branching: 0->1 (0.73/100.0 s, steps/err: 29(193.258047104 ms)/0.0014591646176)
transition motion tried: True
result: 0 - ('track',)->('grip1',) = success
try transition motion
try: 1 - ('grip1',)->('goal',)
error: 2.5
try transition

result: 0 - ('track',)->('grip1',) = fail
++ adding return motion to acquired answer ++
try: 11 - ('grip1',)->('track',)
error: 2.4
try transition motion
Goal reached
Goal reached
transition motion tried: True
result: 11 - ('grip1',)->('track',) = success
branching: 11->21 (2.39/100.0 s, steps/err: 71(136.13319397 ms)/0.00173799219321)
try: 2 - ('grip1',)->('goal',)
transition motion tried: False
error: 2.8
try transition motion
Motion Plan Failure
result: 0 - ('track',)->('grip1',) = fail
try: 2 - ('grip1',)->('goal',)
error: 2.0
try transition motion
transition motion tried: False
Motion Plan Failure
result: 2 - ('grip1',)->('track',) = fail
try: 7 - ('track',)->('grip1',)
error: 2.1
======================= terminated 10: required answers acquired  (2.5/100.0) ===============================
try transition motion
transition motion tried: False
Motion Plan Failure
result: 7 - ('track',)->('grip1',) = fail
======================= terminated 16: Stop called from other agent  (2.5/100.0)

[ERROR] Non-joined subprocesses: [1]
========================== FINISHED (3.6 / 100.0 s) ==============================]
('track',)->('grip1',)
('grip1',)->('goal',)
('goal',)->('goal',)


##### execute plan

In [17]:
crob.reset_connection(True)

connection command:
panda1: True
rospy.init_node() has already been called with different arguments: ('task_planner', ['/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py', '-f', '/home/rnb/.local/share/jupyter/runtime/kernel-2e9edae0-1100-47f3-83a1-c67b02756b91.json'], True, None, False, True)


In [18]:
safe_schedule = calculate_safe_schedule(pscene, schedule, vel_lims=0.2, acc_lims=0.5)

In [20]:
ppline.execute_schedule(safe_schedule)

True

## 5.3 Indy&Panda

##### initialize CombinedRobot and GeometryScene
* Prepare the robots as instructed in release/1.Robots
* Connect gripper to indy - match the direction of grippper with the model added below
* Prepare Kinect and Realsense

In [32]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

from pkg.controller.combined_robot import CombinedRobot
from pkg.controller.robot_config import RobotConfig, RobotType
INDY_IP = "192.168.21.6"
PANDA_HOST_IP = "192.168.21.14" ## The host computer is the computer that runs control program
PANDA_ROBOT_IP = "192.168.21.13" ## The robot has it's own IP

crob = CombinedRobot(
    robots_on_scene=[
        RobotConfig(0, RobotType.indy7gripper, ((0,-0.3,0), (0,0,0)),
                    INDY_IP),
        RobotConfig(1, RobotType.panda, ((0,0.3,0), (0,0,0)),
                    "{}/{}".format(PANDA_HOST_IP, PANDA_ROBOT_IP))
    ], 
    connection_list=[False, False])

xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
ptems = s_builder.add_poles(
    {"cam0": stereo.ref_coord_inv[:3,3], 
     "cam1":np.matmul(stereo.ref_coord_inv, stereo.T_c12)[:3,3]},
    color=(0.6,0.6,0.6,0.2))

gscene.show_pose(crob.home_pose)

connection command:
indy0: False
panda1: False
rospy.init_node() has already been called with different arguments: ('task_planner', ['/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py', '-f', '/home/rnb/.local/share/jupyter/runtime/kernel-2e9edae0-1100-47f3-83a1-c67b02756b91.json'], True, None, False, True)
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


##### add geometries

In [33]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])
track = gtem_dict["track"]

gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])
s_builder.give_clearance(track, gtem_dict.values())

##### create PlanningScene

In [34]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

##### create binders

In [35]:
from pkg.geometry.geometry import *
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="grip1", gname="grip1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="track", gname="track", _type=PlacePlane, point=None)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=(0,0,0.005))

##### create_subject

In [36]:
from pkg.planning.constraint.constraint_subject import Grasp2Point, PlacePoint
from pkg.planning.constraint.constraint_subject import CustomObject, BoxObject

In [37]:
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=BoxObject)

##### create PlanningPipeline

In [39]:
from pkg.planning.pipeline import PlanningPipeline
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.task.rrt import TaskRRT

ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(MoveitPlanner(pscene, enable_dual=False))
ppline.set_task_planner(TaskRRT(pscene))

##### update state

In [40]:
initial_state = pscene.initialize_state(crob.home_pose)
print(initial_state.node)

('track',)


##### search for a plan

In [41]:
ppline.search(initial_state, goal_nodes=[("goal",)], verbose=True, 
              display=False, dt_vis=0.01, timeout_loop=100, multiprocess=True, timeout=1, timeout_constrained=10)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
ppline.play_schedule(schedule, period=0.05)

Use 18/36 agents
try: 0 - ('track',)->('grip0',)
try: 0 - ('track',)->('grip0',)
error: 2.3
error: 2.5
try transition motion
try transition motion
try: 0 - ('track',)->('grip1',)
error: 2.6
try transition motion
try: 0 - ('track',)->('grip1',)
error: 2.9
try: 0 - ('track',)->('grip0',)
try transition motion
error: 2.6
try: 0 - ('track',)->('grip0',)
error: 1.6
try transition motion
try: 0 - ('track',)->('grip1',)
try transition motion
error: 2.5
try transition motion
try: 0 - ('track',)->('grip1',)
error: 1.4
try: 0 - ('track',)->('grip0',)
try transition motion
error: 2.3
try transition motion
try: 0 - ('track',)->('grip0',)
try: 0 - ('track',)->('grip0',)
error: 2.4
error: 2.9
try transition motion
try transition motion
try: 0 - ('track',)->('grip0',)
try: 0 - ('track',)->('grip0',)
transition motion tried: True
error: 0.75
error: 2.5
try transition motion
try transition motion
result: 0 - ('track',)->('grip0',) = success
try: 1 - ('grip0',)->('goal',)
error: 2.0
try transition motio

Process Process-28:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/home/rnb/Projects/rnb-planning/src/pkg/planning/pipeline.py", line 216, in __search_loop
    **kwargs)
  File "/home/rnb/Projects/rnb-planning/src/pkg/planning/pipeline.py", line 290, in test_connection
    self.mplan.plan_transition(from_state, to_state, verbose=verbose, **kwargs)
  File "/home/rnb/Projects/rnb-planning/src/pkg/planning/motion/interface.py", line 153, in plan_transition
    verbose=verbose, **kwargs)
  File "/home/rnb/Projects/rnb-planning/src/pkg/planning/motion/moveit/moveit_planner.py", line 263, in plan_algorithm
    group_name_handle, group_name_binder, subject_list)))
RuntimeError: dual arm motion is not enabled - indy0 & panda1 - ['box1']


transition motion tried: True
Goal reached
result: 1 - ('grip0',)->('goal',) = success
branching: 1->16 (1.76/100.0 s, steps/err: 23(100.734949112 ms)/0.00120101520012)
++ adding return motion to acquired answer ++
try: 3 - ('grip1',)->('grip0',)


Process Process-40:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/home/rnb/Projects/rnb-planning/src/pkg/planning/pipeline.py", line 216, in __search_loop
    **kwargs)
  File "/home/rnb/Projects/rnb-planning/src/pkg/planning/pipeline.py", line 290, in test_connection
    self.mplan.plan_transition(from_state, to_state, verbose=verbose, **kwargs)
  File "/home/rnb/Projects/rnb-planning/src/pkg/planning/motion/interface.py", line 153, in plan_transition
    verbose=verbose, **kwargs)
  File "/home/rnb/Projects/rnb-planning/src/pkg/planning/motion/moveit/moveit_planner.py", line 263, in plan_algorithm
    group_name_handle, group_name_binder, subject_list)))
RuntimeError: dual arm motion is not enabled - panda1 & indy0 - ['box1']


Goal reached
try: 3 - ('grip1',)->('track',)
error: 2.6
try transition motion
try: 10 - ('grip0',)->('grip1',)
transition motion tried: False
Motion Plan Failure
result: 0 - ('track',)->('grip0',) = fail
try: 3 - ('grip1',)->('track',)
error: 2.6
try transition motion


Process Process-32:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/home/rnb/Projects/rnb-planning/src/pkg/planning/pipeline.py", line 216, in __search_loop
    **kwargs)
  File "/home/rnb/Projects/rnb-planning/src/pkg/planning/pipeline.py", line 290, in test_connection
    self.mplan.plan_transition(from_state, to_state, verbose=verbose, **kwargs)
  File "/home/rnb/Projects/rnb-planning/src/pkg/planning/motion/interface.py", line 153, in plan_transition
    verbose=verbose, **kwargs)
  File "/home/rnb/Projects/rnb-planning/src/pkg/planning/motion/moveit/moveit_planner.py", line 263, in plan_algorithm
    group_name_handle, group_name_binder, subject_list)))
RuntimeError: dual arm motion is not enabled - indy0 & panda1 - ['box1']


Goal reached
transition motion tried: True
result: 0 - ('track',)->('grip1',) = success
branching: 0->19 (1.91/100.0 s, steps/err: 53(243.578910828 ms)/0.00183170257922)
try: 19 - ('grip1',)->('goal',)
error: 2.3
try transition motion
try: 6 - ('grip0',)->('goal',)
error: 2.8
try transition motion
transition motion tried: True
Goal reached
result: 6 - ('grip0',)->('goal',) = success
branching: 6->20 (2.08/100.0 s, steps/err: 44(73.4128952026 ms)/0.00209384445786)
++ adding return motion to acquired answer ++
======================= terminated 13: required answers acquired  (2.1/100.0) ===============================
transition motion tried: False
Motion Plan Failure
result: 0 - ('track',)->('grip0',) = fail
======================= terminated 12: Stop called from other agent  (2.1/100.0) ===============================
Goal reached
transition motion tried: False
Motion Plan Failure
result: 0 - ('track',)->('grip0',) = fail
======================= terminated 14: Stop called from other ag

##### execute plan

In [43]:
crob.reset_connection(True, True)

connection command:
indy0: True
panda1: True
rospy.init_node() has already been called with different arguments: ('task_planner', ['/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py', '-f', '/home/rnb/.local/share/jupyter/runtime/kernel-2e9edae0-1100-47f3-83a1-c67b02756b91.json'], True, None, False, True)


In [44]:
safe_schedule = calculate_safe_schedule(pscene, schedule, vel_lims=0.2, acc_lims=0.5)

In [45]:
ppline.execute_schedule(safe_schedule)

Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)


True